In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')



df = pd.read_csv('data\exams.csv')

ModuleNotFoundError: No module named 'xgboost'

In [9]:
!pip install catboost

  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   - -------------------------------------- 4.2/102.5 MB 22.9 MB/s eta 0:00:05
   ---- ----------------------------------- 10.5/102.5 MB 25.1 MB/s eta 0:00:04
   ------ --------------------------------- 16.8/102.5 MB 26.4 MB/s eta 0:00:04
   -------- ------------------------------- 22.0/102.5 MB 26.3 MB/s eta 0:00:04
   --------- ------------------------------ 25.2/102.5 MB 25.3 MB/s eta 0:00:04
   ----------- ---------------------------- 29.9/102.5 MB 23.4 MB/s eta 0:00:04
   ------------ --------------------------- 32.5/102.5 MB 22.2 MB/s eta 0:00:04
   --------------- ------------------------ 38.8/102.5 MB 23.0 MB/s eta 0:00:03
   ----------------- ---------------------- 44.0/102.5 MB 23.0 MB/s eta 0:00:03
   ------------------- -------------------- 49.3/102.5 MB 23.4 MB/s eta 0:00:03
   -------------------- ------------------- 53.5/102.5 MB 23.2 MB/s

In [11]:
!pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ----- ---------------------------------- 10.0/72.0 MB 51.9 MB/s eta 0:00:02
   ----------- ---------------------------- 19.9/72.0 MB 50.4 MB/s eta 0:00:02
   ---------------- ----------------------- 29.9/72.0 MB 49.9 MB/s eta 0:00:01
   ----------------------- ---------------- 41.9/72.0 MB 50.3 MB/s eta 0:00:01
   --------------------------------- ------ 59.8/72.0 MB 57.7 MB/s eta 0:00:01
   -------------------------------------- - 69.2/72.0 MB 57.3 MB/s eta 0:00:01
   ---------------------------------------- 72.0/72.0 MB 49.4 MB/s  0:00:01


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB
